In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline



In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rifatalam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('/Volumes/New Volume/UIU/UIU/5th Semester/ML Project/Project 2.0/training_set.csv')

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,tags,post
0,71058,502503,Money service,When XXXX XXXX closed my account ( account # e...
1,129885,2096726,Consumer Loan,I purchased a vehicle thru XXXX ( XXXX XXXX XX...
2,95306,1277831,Student loan,Sure I called the company and no one could giv...
3,17082,124528,Credit reporting,Equifax fail to delete wrong information on my...
4,116918,2030087,Credit card,I have XXXX Bank of America credit cards. For ...


In [5]:
df = df.drop(columns = 'Unnamed: 0.1', axis = 1)

In [6]:
df = df.dropna()

In [7]:
df = df[pd.notnull(df['tags'])]
print(df.head(10))
print(df['post'].apply(lambda x: len(x.split(' '))).sum())

   Unnamed: 0              tags  \
0       71058     Money service   
1      129885     Consumer Loan   
2       95306      Student loan   
3       17082  Credit reporting   
4      116918       Credit card   
5       52213          Mortgage   
6       21830   Debt collection   
7      122757       Credit card   
8      117831      Bank account   
9       29347          Mortgage   

                                                post  
0  When XXXX XXXX closed my account ( account # e...  
1  I purchased a vehicle thru XXXX ( XXXX XXXX XX...  
2  Sure I called the company and no one could giv...  
3  Equifax fail to delete wrong information on my...  
4  I have XXXX Bank of America credit cards. For ...  
5  I have sent two letters to Plaza Mortgage, dat...  
6  I am the co-signer on my daughter 's auto loan...  
7  Capital one has been charging interest for yea...  
8  According to a number of recent articles relea...  
9  This complaint is in regards to Ocwen 's handl...  
18574374


In [8]:
df['tags'].value_counts()

Bank account        8077
Money service       8042
Debt collection     8039
Student loan        8018
Vehicle loan        8014
Mortgage            7989
Checking account    7980
Credit reporting    7977
Credit card         7905
Consumer Loan       7527
Name: tags, dtype: int64

In [9]:
my_tags = ['Bank account','Money service','Debt collection','Student loan','Vehicle loan','Mortgage','Checking account','Credit reporting','Credit card','Consumer Loan']

In [10]:
def print_plot(index):
    example = df[['post', 'tags']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

When XXXX XXXX closed my account ( account # ending with XXXX ) in XX/XX/XXXX, they decided to return the overpayment amount of my credit card of {$1200.00} electronically back to CheckFree Pay/Fiserv instead of sending me a refund check on XX/XX/XXXX. CheckFree Pay/Fiserv advised XXXX XXXX that I would be required to return to the location where I originally made the overpayment ( XXXX ) in XX/XX/XXXX to collect my refund and provided the trace number XXXX. But when I went back to the original payment location ( XXXX ) to collect my refund, none of the XXXX representative ( including the manager ) recognized the trace number and told me to contact CheckFree Pay/Fiserv. On XX/XX/XXXX, I then contacted CheckFree Pay/Fiserv External Customer Service Department at XXXX to address this issue and even confirmed with XXXX Customer Service Department about the refund of {$1200.00} as per CheckFree Pay/Fiserv protocol. On XX/XX/XXXX, I followed up with CheckFree Pay/Fiserv and escalated this i

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['post'] = df['post'].apply(clean_text)
print_plot(10)

xxxx xxxx closed account account # ending xxxx xx xx xxxx decided return overpayment amount credit card 120000 electronically back checkfree pay fiserv instead sending refund check xx xx xxxx checkfree pay fiserv advised xxxx xxxx would required return location originally made overpayment xxxx xx xx xxxx collect refund provided trace number xxxx went back original payment location xxxx collect refund none xxxx representative including manager recognized trace number told contact checkfree pay fiserv xx xx xxxx contacted checkfree pay fiserv external customer service department xxxx address issue even confirmed xxxx customer service department refund 120000 per checkfree pay fiserv protocol xx xx xxxx followed checkfree pay fiserv escalated issue internal department ticket # xxxx xx xx xxxx followed checkfree pay customer service manager told still resolution unable locate refund xx xx xxxx followed checkfree pay fiserv customer service manager told thing even tried contact xxxx xxxx so

In [12]:
df['post'].apply(lambda x: len(x.split(' '))).sum()

9497420

In [13]:
X = df.post
y = df.tags
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.7507016882409618
                  precision    recall  f1-score   support

    Bank account       0.69      0.65      0.67      2488
   Money service       0.63      0.72      0.68      2414
 Debt collection       0.78      0.34      0.47      2253
    Student loan       0.74      0.77      0.76      2351
    Vehicle loan       0.76      0.82      0.79      2385
        Mortgage       0.79      0.67      0.73      2433
Checking account       0.83      0.80      0.82      2369
Credit reporting       0.80      0.95      0.87      2414
     Credit card       0.88      0.91      0.90      2374
   Consumer Loan       0.65      0.85      0.74      2390

        accuracy                           0.75     23871
       macro avg       0.76      0.75      0.74     23871
    weighted avg       0.76      0.75      0.74     23871



In [51]:
nb_tuned = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB(alpha = 0.33335,fit_prior=False)),
              ])
nb_tuned.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred2 = nb_tuned.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred2, y_test))
print(classification_report(y_test, y_pred2,target_names=my_tags))

accuracy 0.7608814042143186
                  precision    recall  f1-score   support

    Bank account       0.69      0.65      0.67      2488
   Money service       0.64      0.71      0.67      2414
 Debt collection       0.71      0.47      0.57      2253
    Student loan       0.74      0.77      0.76      2351
    Vehicle loan       0.76      0.83      0.79      2385
        Mortgage       0.78      0.70      0.74      2433
Checking account       0.83      0.81      0.82      2369
Credit reporting       0.84      0.94      0.89      2414
     Credit card       0.90      0.91      0.90      2374
   Consumer Loan       0.71      0.81      0.76      2390

        accuracy                           0.76     23871
       macro avg       0.76      0.76      0.76     23871
    weighted avg       0.76      0.76      0.76     23871



In [15]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)



y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.7110720120648486
                  precision    recall  f1-score   support

    Bank account       0.72      0.52      0.61      2488
   Money service       0.73      0.48      0.58      2414
 Debt collection       0.80      0.12      0.20      2253
    Student loan       0.65      0.83      0.73      2351
    Vehicle loan       0.72      0.82      0.76      2385
        Mortgage       0.68      0.77      0.72      2433
Checking account       0.69      0.88      0.77      2369
Credit reporting       0.80      0.95      0.87      2414
     Credit card       0.80      0.93      0.86      2374
   Consumer Loan       0.63      0.79      0.70      2390

        accuracy                           0.71     23871
       macro avg       0.72      0.71      0.68     23871
    weighted avg       0.72      0.71      0.68     23871



In [16]:
sgd_tuned = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='modified_huber', penalty='l2',alpha=0.00001, random_state=0, max_iter=40, tol=None)),
               ])
sgd_tuned.fit(X_train, y_train)



y_pred_tuned = sgd_tuned.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred_tuned, y_test))
print(classification_report(y_test, y_pred_tuned,target_names=my_tags))

accuracy 0.7758786812450253
                  precision    recall  f1-score   support

    Bank account       0.68      0.66      0.67      2488
   Money service       0.69      0.67      0.68      2414
 Debt collection       0.68      0.60      0.64      2253
    Student loan       0.76      0.80      0.78      2351
    Vehicle loan       0.83      0.84      0.83      2385
        Mortgage       0.73      0.77      0.75      2433
Checking account       0.81      0.82      0.81      2369
Credit reporting       0.90      0.92      0.91      2414
     Credit card       0.92      0.91      0.91      2374
   Consumer Loan       0.75      0.77      0.76      2390

        accuracy                           0.78     23871
       macro avg       0.77      0.78      0.77     23871
    weighted avg       0.77      0.78      0.77     23871



In [17]:
!pip install pickle-mixin

In [18]:
import pickle

In [19]:
filename_SDG = 'SDG.sav'
pickle.dump(sgd,open(filename_SDG, 'wb'))

In [20]:
filename_SDG2 = 'SDG_tuned.sav'
pickle.dump(sgd_tuned,open(filename_SDG2, 'wb'))

In [21]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1e5,max_iter = 200)),
               ])
logreg.fit(X_train, y_train)



y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.7376314356331951
                  precision    recall  f1-score   support

    Bank account       0.66      0.63      0.65      2488
   Money service       0.64      0.65      0.65      2414
 Debt collection       0.61      0.61      0.61      2253
    Student loan       0.72      0.73      0.73      2351
    Vehicle loan       0.81      0.79      0.80      2385
        Mortgage       0.66      0.72      0.69      2433
Checking account       0.78      0.78      0.78      2369
Credit reporting       0.88      0.87      0.87      2414
     Credit card       0.90      0.87      0.88      2374
   Consumer Loan       0.74      0.72      0.73      2390

        accuracy                           0.74     23871
       macro avg       0.74      0.74      0.74     23871
    weighted avg       0.74      0.74      0.74     23871



In [22]:
from sklearn.linear_model import LogisticRegression

logreg_tuned = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=250)),
               ])
logreg_tuned.fit(X_train, y_train)



y_pred = logreg_tuned.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

[LibLinear]accuracy 0.7919651459930459
                  precision    recall  f1-score   support

    Bank account       0.72      0.69      0.70      2488
   Money service       0.73      0.69      0.71      2414
 Debt collection       0.70      0.62      0.66      2253
    Student loan       0.77      0.83      0.80      2351
    Vehicle loan       0.83      0.84      0.84      2385
        Mortgage       0.74      0.79      0.76      2433
Checking account       0.82      0.84      0.83      2369
Credit reporting       0.92      0.93      0.92      2414
     Credit card       0.93      0.91      0.92      2374
   Consumer Loan       0.76      0.79      0.77      2390

        accuracy                           0.79     23871
       macro avg       0.79      0.79      0.79     23871
    weighted avg       0.79      0.79      0.79     23871



In [23]:
filename_LOGREG = 'logreg.sav'
pickle.dump(logreg,open(filename_LOGREG, 'wb'))

In [24]:
filename_LOGREG2 = 'logreg_tuned.sav'
pickle.dump(logreg_tuned,open(filename_LOGREG2, 'wb'))

In [25]:
df_val = pd.read_csv('/Volumes/New Volume/UIU/UIU/5th Semester/ML Project/Project 2.0/valiation_set.csv')

In [26]:
df_val.head()

,Unnamed: 0,Unnamed: 0.1,tags,post
0,3,66,Mortgage,I refinanced my home loan with NationStar XX/X...
1,19,201,Mortgage,"On XX/XX/XXXX, I had a rate lock with XXXX XXX..."
2,24,244,Student loan,My Demand Draft Fraud Complaint Against XXXX X...
3,40,355,Debt collection,I OBTAINED A CREDIT CARD FROM XXXX XXXX XXXX ...
4,46,395,Debt collection,I have been a victim of identity theft since 2...


In [27]:
df_val = df_val.drop(columns = 'Unnamed: 0', axis = 1)

In [28]:
df_val = df_val.drop(columns = 'Unnamed: 0.1', axis = 1)

In [29]:
df_val = df_val.dropna()

In [30]:
df_val.describe()

,tags,post
count,19892,19892
unique,10,19781
top,Credit card,I was looking through my report and noticed a ...
freq,2094,14


In [31]:
df_val['tags'].value_counts()

Credit card         2094
Credit reporting    2022
Checking account    2019
Mortgage            2010
Vehicle loan        1985
Student loan        1981
Debt collection     1960
Money service       1957
Consumer Loan       1942
Bank account        1922
Name: tags, dtype: int64

In [32]:
val_tags = ['Credit reporting','Debt collection','Mortgage','Credit card','Checking account','Money service','Bank account','Vehicle loan','Consumer Loan','Student loan']

In [33]:
def print_plot(index):
    example = df_val[['post', 'tags']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

I refinanced my home loan with NationStar XX/XX/XXXX. Upon closing, without notifying me or bringing it to my attention they removed my Flood insurance from my escrow account ** ( per CFPB they have to disclose this information at closing. They never did ). In XX/XX/XXXX my flood insurance was due and did not get paid. I received notice of cancellation and immediately called Nationstar. They told me it was not a part of my escrow and if i wanted them to pay it I needed to talk to the escrow dept. So they transferred me to Escrow. I asked for them to pay my flood insurance and they said they would. * ( this didn't go smoothly there was a lot of back and forth ). Two months later my flood policy was cancelled due to non payment. Then I received a letter from Nationstar telling me they purchased lender placed Flood Insurance for me, which was 3 times the amount of my policy thru XXXX. According to my research with CFPB Subject to the requirements of 1024.37 ( c ) ( 1 ) ( i ) through ( iii

In [34]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df_val['post'] = df_val['post'].apply(clean_text)
print_plot(10)

refinanced home loan nationstar xx xx xxxx upon closing without notifying bringing attention removed flood insurance escrow account per cfpb disclose information closing never xx xx xxxx flood insurance due get paid received notice cancellation immediately called nationstar told part escrow wanted pay needed talk escrow dept transferred escrow asked pay flood insurance said would didnt go smoothly lot back forth two months later flood policy cancelled due non payment received letter nationstar telling purchased lender placed flood insurance 3 times amount policy thru xxxx according research cfpb subject requirements 102437 c 1 iii prohibited state applicable law servicer may charge borrower forceplaced insurance servicer purchased retroactive first day period time borrower hazard insurance place insurance spoke xxxx could believe lost grandfathered rate 140000 annually insurance 370000 mess made escrow account low raised monthly payments 50000 monthso high single mom xxxx growing child

In [35]:
X_val = df_val.post
y_val = df_val.tags


In [36]:
loaded_model_LOGREG = pickle.load(open(filename_LOGREG, 'rb'))
result = loaded_model_LOGREG.score(X_val,y_val)
print(result)
y_pred_LOGREG = loaded_model_LOGREG.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred_LOGREG, y_val))
print(classification_report(y_pred_LOGREG, y_val,target_names=val_tags))

0.7377840337824251
accuracy 0.7377840337824251
                  precision    recall  f1-score   support

Credit reporting       0.63      0.64      0.64      1904
 Debt collection       0.67      0.64      0.66      2107
        Mortgage       0.61      0.61      0.61      1936
     Credit card       0.74      0.75      0.74      2054
Checking account       0.78      0.80      0.79      1970
   Money service       0.73      0.67      0.70      2129
    Bank account       0.77      0.78      0.77      1953
    Vehicle loan       0.86      0.88      0.87      1965
   Consumer Loan       0.86      0.90      0.88      1903
    Student loan       0.72      0.73      0.72      1971

        accuracy                           0.74     19892
       macro avg       0.74      0.74      0.74     19892
    weighted avg       0.74      0.74      0.74     19892



In [37]:
loaded_model_LOGREG2 = pickle.load(open(filename_LOGREG2, 'rb'))
result = loaded_model_LOGREG2.score(X_val,y_val)
print(result)
y_pred_LOGREG2 = loaded_model_LOGREG2.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred_LOGREG2, y_val))
print(classification_report(y_pred_LOGREG2, y_val,target_names=val_tags))

0.7951940478584355
accuracy 0.7951940478584355
                  precision    recall  f1-score   support

Credit reporting       0.69      0.70      0.69      1891
 Debt collection       0.69      0.73      0.71      1896
        Mortgage       0.62      0.71      0.66      1680
     Credit card       0.83      0.79      0.81      2180
Checking account       0.85      0.82      0.84      2091
   Money service       0.81      0.75      0.78      2125
    Bank account       0.84      0.81      0.83      2035
    Vehicle loan       0.92      0.91      0.92      2037
   Consumer Loan       0.90      0.93      0.92      1919
    Student loan       0.79      0.77      0.78      2038

        accuracy                           0.80     19892
       macro avg       0.79      0.79      0.79     19892
    weighted avg       0.80      0.80      0.80     19892



In [38]:
loaded_model_SDG = pickle.load(open(filename_SDG, 'rb'))
result = loaded_model_SDG.score(X_val,y_val)
print(result)

y_pred_SDG = loaded_model_SDG.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred_SDG, y_val))
print(classification_report(y_pred_SDG, y_val,target_names=val_tags))

0.7127991152221999
accuracy 0.7127991152221999
                  precision    recall  f1-score   support

Credit reporting       0.52      0.71      0.60      1422
 Debt collection       0.50      0.74      0.60      1369
        Mortgage       0.11      0.81      0.20       271
     Credit card       0.83      0.69      0.75      2509
Checking account       0.83      0.72      0.77      2330
   Money service       0.78      0.68      0.72      2252
    Bank account       0.88      0.68      0.77      2552
    Vehicle loan       0.95      0.80      0.87      2395
   Consumer Loan       0.92      0.79      0.85      2314
    Student loan       0.77      0.62      0.69      2478

        accuracy                           0.71     19892
       macro avg       0.71      0.72      0.68     19892
    weighted avg       0.79      0.71      0.74     19892



In [39]:
loaded_model_SDG2 = pickle.load(open(filename_SDG2, 'rb'))
result = loaded_model_SDG2.score(X_val,y_val)
print(result)

y_pred_SDG2 = loaded_model_SDG2.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred_SDG2, y_val))
print(classification_report(y_pred_SDG2, y_val,target_names=val_tags))

0.7767444198672834
accuracy 0.7767444198672834
                  precision    recall  f1-score   support

Credit reporting       0.67      0.66      0.67      1954
 Debt collection       0.66      0.69      0.68      1925
        Mortgage       0.59      0.69      0.64      1666
     Credit card       0.79      0.79      0.79      2091
Checking account       0.85      0.82      0.83      2096
   Money service       0.79      0.73      0.76      2139
    Bank account       0.82      0.81      0.81      1984
    Vehicle loan       0.92      0.89      0.90      2063
   Consumer Loan       0.89      0.92      0.91      1935
    Student loan       0.77      0.75      0.76      2039

        accuracy                           0.78     19892
       macro avg       0.78      0.78      0.77     19892
    weighted avg       0.78      0.78      0.78     19892



In [40]:
y_pred = loaded_model_LOGREG.predict(X_val)